In [1]:
# 데이터 불러오기
import pandas as pd
import re 

pd.set_option('display.max_rows', 10000)

df = pd.read_csv(
    "./data/202512_202512_연령별인구현황_월간.csv",
    encoding="cp949"
)

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype("Int64")

# 3. 시/군 단위만 추출
# - 시도: code % 100_000_000 == 0
# - 시군구: code % 100_000_000 != 0 AND code % 100_000 == 0
sigungu = df[
    (df["행정코드"] % 100_000_000 != 0) &
    (df["행정코드"] % 100_000 == 0)
].copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2025년12월_계_총인구수",
    "2025년12월_계_20~24세",
    "2025년12월_계_25~29세",
    "2025년12월_계_30~34세",
]

for c in num_cols:
    sigungu[c] = (
        sigungu[c]
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(int)
    )

# 5. 청년(20~34) 인구 계산
sigungu["청년(20~34)인구"] = (
    sigungu["2025년12월_계_20~24세"]
    + sigungu["2025년12월_계_25~29세"]
    + sigungu["2025년12월_계_30~34세"]
)

sigungu["청년비율(20~34)"] = (
    sigungu["청년(20~34)인구"]
    / sigungu["2025년12월_계_총인구수"]
)

# 6. 제외할 광역 단위 정의
exclude_regions = [
    "서울특별시", "경기도", 
    "인천광역시", "광주광역시", "울산광역시",
    "부산광역시", "대구광역시", "대전광역시"
]

sigungu["상위지역"] = sigungu["지역명"].str.split().str[0]

local_sigungu = sigungu[
    ~sigungu["상위지역"].isin(exclude_regions)
].copy()

# 7. 결과 테이블 구성
result = local_sigungu[
    ["지역명", "2025년12월_계_총인구수", "청년(20~34)인구", "청년비율(20~34)"]
].rename(columns={"2025년12월_계_총인구수": "총인구"})

result["청년비율(20~34)"] = (result["청년비율(20~34)"] * 100).round(2)

# 8. 결과 출력
print("=== [지방 시/군] 청년비율(20~34) 높은 지역 TOP 20 ===")
display(result.sort_values("청년비율(20~34)", ascending=False).head(20))

print("=== [지방 시/군] 청년인구(20~34) 많은 지역 TOP 20 ===")
display(result.sort_values("청년(20~34)인구", ascending=False).head(20))

=== [지방 시/군] 청년비율(20~34) 높은 지역 TOP 20 ===


,지역명,총인구,청년(20~34)인구,청년비율(20~34)
169,충청북도 청주시 흥덕구,292051,63608,21.78
184,충청남도 천안시 서북구,401852,86596,21.55
182,충청남도 천안시,664322,136935,20.61
168,충청북도 청주시 서원구,182689,36320,19.88
166,충청북도 청주시,856152,168807,19.72
170,충청북도 청주시 청원구,186861,36270,19.41
183,충청남도 천안시 동남구,262470,50339,19.18
267,경상남도 창원시 성산구,246904,47210,19.12
147,강원특별자치도 춘천시,285234,53555,18.78
201,전북특별자치도 전주시 완산구,316419,59303,18.74


=== [지방 시/군] 청년인구(20~34) 많은 지역 TOP 20 ===


,지역명,총인구,청년(20~34)인구,청년비율(20~34)
166,충청북도 청주시,856152,168807,19.72
265,경상남도 창원시,990898,159316,16.08
182,충청남도 천안시,664322,136935,20.61
200,전북특별자치도 전주시,625437,116554,18.64
275,경상남도 김해시,533035,87433,16.40
184,충청남도 천안시 서북구,401852,86596,21.55
292,제주특별자치도 제주시,485182,82843,17.07
246,경상북도 구미시,403883,72973,18.07
240,경상북도 포항시,488707,72539,14.84
86,세종특별자치시,391965,63750,16.26
